In [1]:
%reload_ext autoreload
%load_ext autoreload
%autoreload 2
import os
import pandas as pd
import sqlalchemy as sa
import sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('main.ipynb'))))

from tables import County, DataTrustedIdentifier, TrustedIdentifier, DataSP, InfoImage, create_data_trusted_identifier
from database import connect, create_table_if_not_exists, table_is_empty, \
    insert_new_identifier_trusted, find_and_replace_broken_characters, \
    insert_new_data_trusted_identifier, update_country_trusted_based_original_field, has_brasil_in_country_trusted, \
    update_country_trusted, get_all_identifiers_ilike, execute_query

from unaccent import unaccent

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## database connection

In [2]:
engine, session = connect()
engine.echo=False

2023-03-19 23:37:04,241 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-03-19 23:37:04,242 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-19 23:37:04,243 INFO sqlalchemy.engine.Engine select current_schema()
2023-03-19 23:37:04,243 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-19 23:37:04,244 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-03-19 23:37:04,245 INFO sqlalchemy.engine.Engine [raw sql] {}


## list with full name of identifier

In [3]:
list_identifier_trusted = {
    'full_name': ['Aline Vieira de Melo Silva', 'Carmen Lúcia Falcão Ichaso', 'Daniele Monteiro Ferreira', 'Daniel Ruschel', 'Elsie Franklin Guimarães', 'Eric J Tepe', 'Erika Erika Von Sohsten de Souza Medeiros', 'George Azevedo de Queiroz', 'Micheline Carvalho-Silva', 'Ricardo de la Merced Callejas Posada', 'Truman George Yuncker', 'William Trelease'],
    'searched_name': ['Silva', 'Ichaso', 'Monteiro', 'Ruschel', 'Guimar', 'Tepe', 'Medeiros', 'Queiroz', 'Carvalho', 'Callejas', 'Yuncker', 'Trelease']
}

pd.DataFrame(list_identifier_trusted)

,full_name,searched_name
0,Aline Vieira de Melo Silva,Silva
1,Carmen Lúcia Falcão Ichaso,Ichaso
2,Daniele Monteiro Ferreira,Monteiro
3,Daniel Ruschel,Ruschel
4,Elsie Franklin Guimarães,Guimar
5,Eric J Tepe,Tepe
6,Erika Erika Von Sohsten de Souza Medeiros,Medeiros
7,George Azevedo de Queiroz,Queiroz
8,Micheline Carvalho-Silva,Carvalho
9,Ricardo de la Merced Callejas Posada,Callejas


## trusted identifiers

In [4]:
create_table_if_not_exists(engine, TrustedIdentifier)

count_of_records_trusted_identifier = session.query(TrustedIdentifier) \
                                        .count()

if count_of_records_trusted_identifier == 0:
    list_full_name = list_identifier_trusted['full_name']
    list_searched_name = list_identifier_trusted['searched_name']
    for item in zip(list_full_name, list_searched_name):
        full_name_identifier = item[0]
        variation_of_name_identfier = item[1]

        query = session.query(DataSP.identified_by) \
            .filter(DataSP.identified_by.ilike('%{}%'.format(variation_of_name_identfier))) \
            .distinct(DataSP.identified_by) \
            .all()

        for q in query:
            print('full name identifier: %s variation founded: %s' % (full_name_identifier, q[0]))
            new_identifier_trusted = TrustedIdentifier(name=full_name_identifier, searched_name=variation_of_name_identfier,
                                                       value_founded=q[0], trusted=False)

            session.add(new_identifier_trusted)

In [7]:
count_variations_identifier_name_marked_trusted = session.query(TrustedIdentifier.value_founded) \
    .filter(TrustedIdentifier.trusted) \
    .distinct()\
    .count()

print('count of varations founded of identifiers name marked trusted: %d' % count_variations_identifier_name_marked_trusted)

count of varations founded of identifiers name marked trusted: 187


## get variations of identfiers trusted

In [8]:
query = session.query(TrustedIdentifier.value_founded) \
    .filter(TrustedIdentifier.trusted) \
    .distinct()\
    .all()

list_variations_of_identifiers_trusted = [q[0] for q in query]

In [12]:
count_of_records_with_variations_identifier_name = session.query(DataSP) \
    .filter(DataSP.identified_by.in_(list_variations_of_identifiers_trusted))\
    .count()

print('count of records founded with variations of identifier name: %d' % count_of_records_with_variations_identifier_name)

count of records founded with variations identifier name: 13182


In [11]:
create_table_if_not_exists(engine, InfoImage)
create_table_if_not_exists(engine, DataTrustedIdentifier)

In [13]:
if table_is_empty(session, DataTrustedIdentifier):
    query = session.query(DataSP) \
        .filter(DataSP.identified_by.in_(list_variations_of_identifiers_trusted)) \
        .all()
    for q in query:
        new_data_of_identifier_trusted = create_data_trusted_identifier(q)
        session.add(new_data_of_identifier_trusted)

count_data_from_trusted_identifers = session.query(DataTrustedIdentifier).count()
print('count of records in table data_trusted_identifier: %d' % count_data_from_trusted_identifers)

count of records in table data_trusted_identifier: 13182


## replace unenconded character

In [14]:
list_unenconded_characters = {
    'error': ['Ã¡', 'Ãº', 'Ã', 'Ã³', 'Ã±', 'Ã©'],
    'correct': ['á', 'ú', 'í', 'ó', 'ñ', 'é']
}

pd.DataFrame(list_unenconded_characters)

,find,replace
0,Ã¡,á
1,Ãº,ú
2,Ã,í
3,Ã³,ó
4,Ã±,ñ
5,Ã©,é


In [6]:
for column in [DataTrustedIdentifier.state_province, DataTrustedIdentifier.county]:
    list_character_error = list_unenconded_characters['error']
    list_character_correct = list_unenconded_characters['correct']
    for special_character in zip(list_character_error, list_character_correct):
        value = sa.func.replace(column, special_character['find'], special_character['replace'])
        session.query(DataTrustedIdentifier)\
            .update(values={column: value}, synchronize_session=False)

## replace variations of word Brazil

In [15]:
count_of_brazil_in_country_trusted = session.query(DataTrustedIdentifier) \
    .filter(DataTrustedIdentifier.country_trusted == 'Brasil') \
    .count()

print('count of Brasil in country trusted: %d' % count_of_brazil_in_country_trusted)

count of Brasil in country trusted: 12144


In [16]:
list_variations_br = ['Brasil', 'BRASIL', 'Brasil/Bolivia', 'Brasilia', 'brazil', 'Brazil', 'BRazil', 'BRAZIL',
                      '[Brésil]',
                      'Brésil']

if count_of_brazil_in_country_trusted == 0:
    records_with_variations_brasil = session.query(DataTrustedIdentifier) \
        .filter(DataTrustedIdentifier.country.in_(list_variations_br)) \
        .all()

    print('count of records with variations of Brazil: %d' % len(records_with_variations_brasil))

    session.query(DataTrustedIdentifier) \
        .filter(DataTrustedIdentifier.country.in_(list_variations_br)) \
        .update({'country_trusted': 'Brasil'}, synchronize_session=False)

## insert BR in column country_trusted

In [9]:
county = session.query(County).distinct().all()

list_uf = [unaccent(sa.func.lower(c.uf)) for c in county]
list_state = [unaccent(sa.func.lower(c.state)) for c in county]
list_county = [unaccent(sa.func.lower(c.county)) for c in county]

uf_unaccent_lower = unaccent(sa.func.lower(DataTrustedIdentifier.state_province)).in_(list_uf)
state_unaccent_lower = unaccent(sa.func.lower(DataTrustedIdentifier.state_province)).in_(list_state)
county_unaccent_lower = unaccent(sa.func.lower(DataTrustedIdentifier.county)).in_(list_county)

session.query(DataTrustedIdentifier) \
    .filter(sa.and_(DataTrustedIdentifier.country_trusted.is_(None),
                 sa.or_(uf_unaccent_lower, state_unaccent_lower),
                 county_unaccent_lower)) \
    .update({'country_trusted': 'Brasil'}, synchronize_session=False)

/home/xandao/Documentos/piperaceae-sql/database.py:185: SAWarning: Class unaccent will not make use of SQL compilation caching as it does not set the 'inherit_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Set this attribute to True if this object can make use of the cache key generated by the superclass.  Alternatively, this attribute may be set to False which will disable this warning. (Background on this error at: https://sqlalche.me/e/20/cprf)
  session.query(DataTrustedIdentifier)\


## close connection

In [8]:
session.close()
engine.dispose()